In [1]:
import pandas as pd
import os

# Define relative paths
INPUT_DIR = "../input"
OUTPUT_DIR = "../build"

# Input and output file paths
routers_file = os.path.join(INPUT_DIR, "sorted_routers_list.txt")
connections_file = os.path.join(INPUT_DIR, "connections_with_serial.txt")
csv_file = os.path.join(INPUT_DIR, "BSNL_MPPLAB_Traffic_March_2025_Subset.csv")
output_dat_file = os.path.join(OUTPUT_DIR, "NetworkInput_BSNL.dat")
output_txt_file = os.path.join(OUTPUT_DIR, "NetworkInput_BSNL.txt")

# Read routers and create mappings
router_to_id = {}
id_to_router = {}
with open(routers_file, "r") as f:
    for line in f:
        parts = line.strip().split(maxsplit=1)
        if len(parts) == 2:
            router_id, router_name = parts
            router_id = int(router_id)
            router_to_id[router_name] = router_id
            id_to_router[router_id] = router_name
num_routers = len(router_to_id)

# Read connections
edges = []
with open(connections_file, "r") as f:
    next(f)  # Skip header
    for line in f:
        r1_id, r2_id = map(int, line.strip().split())
        edges.append((r1_id, r2_id))

# Read CSV file
df = pd.read_csv(csv_file, usecols=["Router1", "Router2", "Cost", "Capacity"])
df = df.dropna(subset=["Router1", "Router2", "Cost", "Capacity"])
df["Cost"] = pd.to_numeric(df["Cost"], errors="coerce")
df["Capacity"] = pd.to_numeric(df["Capacity"], errors="coerce") * 1_000_000_000
df = df.dropna()

# Create cost and capacity lookup
cost_capacity_lookup = {}
for _, row in df.iterrows():
    r1, r2 = row["Router1"], row["Router2"]
    cost, capacity = row["Cost"], row["Capacity"]
    cost_capacity_lookup[(r1, r2)] = (cost, capacity)
    cost_capacity_lookup[(r2, r1)] = (cost, capacity)

# Process edges with cost and capacity
final_edges = []
for r1_id, r2_id in edges:
    r1_name = id_to_router.get(r1_id, "")
    r2_name = id_to_router.get(r2_id, "")
    cost, capacity = cost_capacity_lookup.get((r1_name, r2_name), (0.0, 0))
    final_edges.append((r1_id, r2_id, cost, capacity))
    final_edges.append((r2_id, r1_id, cost, capacity))

# Sort edges
final_edges.sort(key=lambda x: (x[0], x[1]))
num_edges = len(final_edges)

# Write output to both .dat and .txt files in build folder
output_content = f"{num_routers} {num_edges}\n"
for src, dst, cost, capacity in final_edges:
    output_content += f"{src:>4} {dst:>4} {cost:>10.6f} {capacity:>12.0f}\n"

# Write to .dat file
with open(output_dat_file, "w") as f:
    f.write(output_content)

# Write to .txt file
with open(output_txt_file, "w") as f:
    f.write(output_content)